# imshow部分修改

直接按照
+ <https://github.com/open-mmlab/mmsegmentation/blob/dev-1.x/docs/zh_cn/user_guides/visualization.md>
+ <https://github.com/MeowZheng/mmsegmentation/blob/dev-1.x/mmseg/visualization/local_visualizer.py>

给的调用示例，写个demo出来, 
+ local_visualizer里给的demo已经过期了。。。要改
+ 网页文档里的代码没有过期。。。


关于mmcv的imread，源码：<https://github.com/open-mmlab/mmcv/blob/987d34b0cf8d6cd8725258332fcfc8c54529b1ab/mmcv/image/io.py#L33>

## 测试代码

In [1]:
import os
print(os.getcwd())

test_image_dir = "./pseudo_bdd100k_dataset/images/10k/train"  
test_mask_dir = "./pseudo_bdd100k_dataset/labels/sem_seg/masks/train"

image_name = "0004a4c0-d4dff0ad"

image_path = os.path.join(test_image_dir,image_name+".jpg")
mask_path = os.path.join(test_mask_dir,image_name+".png")

/Users/huangshan/Documents/DailyStudy/mmsegmentation/TestPR


```python
import mmcv
import matplotlib.pyplot as plt

image = mmcv.imread(image_path,channel_order='rgb')
# 不加这个参数默认是bgr
print(type(image),image.shape)

# plt.imshow(image[:,:,::-1])
plt.imshow(image)
```

```python
mask = mmcv.imread(mask_path,'unchanged')

import matplotlib
from matplotlib.colors import ListedColormap
import numpy as np
rgb=np.array([[128, 64, 128],[244, 35, 232],[70, 70, 70],[102, 102, 156],[190, 153, 153],[153, 153, 153],[250, 170, 30],[220, 220, 0],
    [107, 142, 35],[152, 251, 152],[70, 130, 180],[220, 20, 60],[255, 0, 0],[0, 0, 142],[0, 0, 70],[0, 60, 100],[0, 80, 100],[0, 0, 230],[119, 11, 32]])/255.0
rgb_c=np.zeros((256,3))
# 0-255,cmap是256种颜色
for i in range(19):
    rgb_c[i]=rgb[i]
print(rgb_c[:2])
bdd_cmap = ListedColormap(rgb_c)

print(type(mask),mask.shape)
plt.imshow(mask,cmap=bdd_cmap,vmax=255)
```

## 规整代码

In [ ]:
import numpy as np
import torch
from mmengine.structures import PixelData

import mmcv

from importlib import reload
from mmseg.structures import SegDataSample
from  mmseg.visualization.local_visualizer import SegLocalVisualizer

# from mmseg.utils.class_names import bdd100k_classes,bdd100k_palette
# from mmseg.utils.class_names import cityscapes_classes,cityscapes_palette

save_dir = './work_dirs'

seg_local_visualizer = SegLocalVisualizer(vis_backends=[dict(type='LocalVisBackend')],
                                          save_dir=save_dir,
                                          dataset_name = "cityscapes")
image = mmcv.imread(image_path,channel_order='rgb')
# SegLocalVisualizer中要求图像是RGB的

sem_seg = mmcv.imread(mask_path,'unchanged')
gt_sem_seg_data = dict(data=sem_seg)
gt_sem_seg = PixelData(**gt_sem_seg_data)

gt_seg_data_sample = SegDataSample()
gt_seg_data_sample.gt_sem_seg = gt_sem_seg


# seg_local_visualizer.dataset_meta = dict(classes=bdd100k_classes(),palette=bdd100k_palette())
# seg_local_visualizer.dataset_meta = dict(classes=cityscapes_classes(),palette=cityscapes_palette())
seg_local_visualizer.add_datasample('visualizer_example',image, gt_seg_data_sample)

In [ ]:
%matplotlib --list

`inline`模式显示不出来，对于mac来说，只有`osx`这种窗口式的才可以
+ <https://matplotlib.org/stable/users/explain/backends.html#what-is-a-backend>

要么就写个.py文件，反正必须是窗口才行

In [ ]:
%matplotlib osx
seg_local_visualizer.add_datasample('visualizer_example', image,gt_seg_data_sample, show=True)

# imwrite部分修改

# SegLocalVisualizer(本地简写)

## 相关草稿

### mask to polygon

其实思路是一样的，都是直接从mask找轮廓
+ 要么就是调用，skimage.measure.find_contours
    + <https://scikit-image.org/docs/stable/auto_examples/edges/plot_contours.html>
    + [convert mask binary image to polygon format #131](https://github.com/cocodataset/cocoapi/issues/131)
+ 要么就是调用, cv2.findContours
    + <https://www.geeksforgeeks.org/find-and-draw-contours-using-opencv-python/>
    + <https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html>
    + <https://docs.opencv.org/3.2.0/dc/da5/tutorial_py_drawing_functions.html>
    
    + [Image segmentation mask to polygon for coco json](https://stackoverflow.com/questions/68663512/image-segmentation-mask-to-polygon-for-coco-json) 
    + <https://stackoverflow.com/questions/58884265/python-convert-binary-mask-to-polygon>

## 整体测试

由于这个MMengine的注册机制，没法reload，所以直接在本地重新copy一遍那个类，简单写一下就行

In [3]:
import numpy as np
import torch
from mmengine.structures import PixelData
import mmcv

from importlib import reload
from mmseg.structures import SegDataSample
from local_visualizer import SegLocalVisualizer
%matplotlib osx
# from mmseg.utils.class_names import bdd100k_classes,bdd100k_palette


save_dir = './work_dirs'

seg_local_visualizer = SegLocalVisualizer(vis_backends=[dict(type='LocalVisBackend')],
                                          save_dir=save_dir,
                                          dataset_name = "cityscapes")
image = mmcv.imread(image_path,channel_order='rgb')
# SegLocalVisualizer中要求图像是RGB的

sem_seg = mmcv.imread(mask_path,'unchanged')
gt_sem_seg_data = dict(data=sem_seg)
gt_sem_seg = PixelData(**gt_sem_seg_data)

gt_seg_data_sample = SegDataSample()
gt_seg_data_sample.gt_sem_seg = gt_sem_seg


# seg_local_visualizer.dataset_meta = dict(classes=bdd100k_classes(),palette=bdd100k_palette())
# seg_local_visualizer.dataset_meta = dict(classes=cityscapes_classes(),palette=cityscapes_palette())
# seg_local_visualizer.add_datasample('visualizer_example',image, gt_seg_data_sample)

seg_local_visualizer.add_datasample('visualizer_example', image,gt_seg_data_sample, show=True)

/Users/huangshan/Documents/software/miniconda3/miniconda3/envs/py37/lib/python3.7/site-packages/mmengine/structures/pixel_data.py:83: UserWarning: The shape of value will convert from (720, 1280) to (1, 720, 1280)
  warnings.warn('The shape of value will convert from '


## 主要函数修改

+ <https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga819779b9857cc2f8601e6526a3a5bc71>
    ```bash
    cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
     # 使用 RETR_EXTERNAL 模式，不使用RETR_TREE模式，后者会返回多个contours
    ```
+ 不管用哪个模式，似乎都会返回多个。。。

In [10]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Patch
from matplotlib.collections import PatchCollection
import cv2
from mmseg.utils.class_names import cityscapes_classes,cityscapes_palette

def get_coutours(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_num = len(contours)
    # print(f"type contours:{type(contours)}, contours: {len(contours)}")
    if contours_num==1:
        contours_np = np.squeeze(contours)
        # print(f"contours: {contours[0][:6]}, \n contours_np: {contours_py.shape}, {type(contours_py)}\n {contours_py[:6]}")
    else:
        contours_list = []
        for num in range(1,contours_num):
            contour = np.squeeze(contours[num],axis=1)
            # 会出现 contour.shape: (1, 2), [[208  12]] 只有1个点的contour，所以不能所有轴都squeeze
            # print(f"contour.shape: {contour.shape}, {contour[:3]}")
            one_contour_list = contour.tolist()
            contours_list.extend(one_contour_list)
            # print(f"contour type: {type(contour)}")
            # contours.extend(contour)
        # print(f"contours_list: {contours_list[:6]}")
        contours_np = np.array(contours_list,dtype=np.uint8)
        # print(f"contours_np: {contours_np[:4]}")
    return contours_np
            
    

def _draw_sem_seg(image, sem_seg,classes,palette,withLabels = False):
    alpha=0.8
    num_classes = len(classes)

    sem_seg = sem_seg.cpu().data
    ids = np.unique(sem_seg)[::-1]
    # print(f"ids:{ids}")
    legal_indices = ids < num_classes
    # print(f"legal_indices:{legal_indices}")
    ids = ids[legal_indices]
    labels = np.array(ids, dtype=np.int64)
    print(f"labels:{labels}")

    colors = [palette[label] for label in labels]
   
    if withLabels:
        fig, ax = plt.subplots()
        mask = np.zeros(image.shape[:2],dtype=np.uint8)
        masks = sem_seg[0] == labels[:, None, None]
        # opencv需要接受uint8的数据类型
        masks = masks.astype(np.uint8)
        # print(f"masks.shape: {masks.shape}") # (10, 720, 1280)
        # plt.imshow(masks[0])
        # print(masks[0].shape)
        color_seg = masks
        polygons = []
        for mask_num in range(len(labels)):
            classes_id = labels[mask_num]
            # print(f"mask.shape: {mask.shape}, mask.dtype:{mask.dtype}")
            contours = get_coutours(masks[mask_num])
            # print(f"num: {mask_num}, contours:{contours[:4]}")
            face_color = np.array(palette[classes_id])/255.0
            polygon = Polygon(contours,color=face_color)
            polygons.append(polygon)
            text = classes[classes_id]
            print(f"face_color = {face_color}, text = {text}, contours[-1] = {contours[-1]}")
            ax.text(contours[1][0],contours[1][1], text, horizontalalignment='center', verticalalignment='center')
        collection = PatchCollection(polygons, linewidths=2)
        ax.add_collection(collection)
        plt.imshow(image) 
    else:
        mask = np.zeros_like(image, dtype=np.uint8)
        for label, color in zip(labels, colors):
            mask[sem_seg[0] == label, :] = color
        color_seg = (image * (1 - alpha) + mask * alpha).astype(np.uint8)
    return color_seg

In [11]:
color_seg = _draw_sem_seg(image, gt_sem_seg,classes=cityscapes_classes(),palette=cityscapes_palette(),withLabels=True)
color_seg.shape

labels:[13 11 10  8  7  6  5  4  2  0]
face_color = [0.         0.         0.55686275], text = car, contours[-1] = [990 115]
face_color = [0.8627451  0.07843137 0.23529412], text = person, contours[-1] = [734 188]
face_color = [0.2745098  0.50980392 0.70588235], text = sky, contours[-1] = [208   0]
face_color = [0.41960784 0.55686275 0.1372549 ], text = vegetation, contours[-1] = [19  0]
face_color = [0.8627451 0.8627451 0.       ], text = traffic sign, contours[-1] = [211  42]
face_color = [0.98039216 0.66666667 0.11764706], text = traffic light, contours[-1] = [98 38]
face_color = [0.6 0.6 0.6], text = pole, contours[-1] = [218   0]
face_color = [0.74509804 0.6        0.6       ], text = fence, contours[-1] = [ 54 175]
face_color = [0.2745098 0.2745098 0.2745098], text = building, contours[-1] = [2 0]
face_color = [0.50196078 0.25098039 0.50196078], text = road, contours[-1] = [115   0]


(10, 720, 1280)

### 全用opencv画

参考：<https://stackoverflow.com/questions/16615662/how-to-write-text-on-a-image-in-windows-using-python-opencv2>

In [6]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Patch
from matplotlib.collections import PatchCollection
import cv2
from mmseg.utils.class_names import cityscapes_classes,cityscapes_palette

def get_loc(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    loc = np.squeeze(contours[0],axis=1)[0]
    return loc
            
    
def _draw_sem_seg(image, sem_seg,classes,palette,withLabels = False):
    alpha=0.8
    num_classes = len(classes)

    sem_seg = sem_seg.cpu().data
    ids = np.unique(sem_seg)[::-1]
    # print(f"ids:{ids}")
    legal_indices = ids < num_classes
    # print(f"legal_indices:{legal_indices}")
    ids = ids[legal_indices]
    labels = np.array(ids, dtype=np.int64)
    print(f"labels:{labels}")

    colors = [palette[label] for label in labels]
   
    if withLabels:
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        fontScale              = 1
        fontColor              = (255,255,255)
        thickness              = 3
        lineType               = 2
        
        mask = np.zeros(image.shape[:2],dtype=np.uint8)
        masks = sem_seg[0] == labels[:, None, None]
        masks = masks.astype(np.uint8)

        for mask_num in range(len(labels)):
            classes_id = labels[mask_num]
            loc = get_loc(masks[mask_num])
            text = classes[classes_id]
            print(f"loc = {loc}, text = {text}")
            image = cv2.putText(image,text, loc, font, fontScale,fontColor,thickness,lineType)
            
        mask = np.zeros_like(image, dtype=np.uint8)
        for label, color in zip(labels, colors):
            mask[sem_seg[0] == label, :] = color
        color_seg = (image * (1 - alpha) + mask * alpha).astype(np.uint8)
        plt.imshow(color_seg) 
    return color_seg

masks = _draw_sem_seg(image, gt_sem_seg,classes=cityscapes_classes(),palette=cityscapes_palette(),withLabels=True)

labels:[13 11 10  8  7  6  5  4  2  0]
loc = [956 115], text = car
loc = [726 188], text = person
loc = [540  95], text = sky
loc = [206 140], text = vegetation
loc = [998 152], text = traffic sign
loc = [1006  192], text = traffic light
loc = [999 161], text = pole
loc = [265 254], text = fence
loc = [621 255], text = building
loc = [1252  406], text = road


### 用matplotlib画

+ <https://medium.com/analytics-vidhya/how-to-plot-image-and-text-on-matplotlib-91d7f23a3043>

### 用PIL画

+ <https://holypython.com/python-pil-tutorial/how-to-add-text-to-images-in-python-via-pil-library/>